In [ ]:
import glob
import os.path

import numpy

import deepometry.model

# Evaluate

After training a model to classify single cell images, it is often useful to evaluate the performance of the model on an unseen annotated dataset. Evaluation helps predict model performance on unseen data.

Suppose we have the following directory structure where images from one experiment (`experiment_02`) have been classified as one of three classes (`class_A`, `class_B`, or `class_C`). Data from this experiment was not shown to the model during training. Images are saved as NPY files:

    /data/
        experiment_02/
            class_A/
                32e88e1ac3a8f44bf8f77371155553b9.npy
                3dc56a0c446942aa0da170acfa922091.npy  
                ...
            class_B/
                8068ef7dcddd89da4ca9740bd2ccb31e.npy
                8348deaa70dfc95c46bd02984d28b873.npy
                ...
            class_C/  
                c1ecbca7bd98c01c1d3293b64cd6739a.npy
                c56cfb8e7e7121dd822e47c67d07e2d4.npy
                ...
                

The data can be used to evaluate a model for classifying image data as one of the three classes. The `collect_pathnames` and `load` functions defined below will select images to use for evaluating the model and generate the labels for the evaluation images.

In [ ]:
def collect_pathnames(directories):
    """
    :param directories: List of directories to select samples from. Assumes subdirectories of each directory
                        correspond to class labels. Contents of subdirectories are NPY files containing data
                        of that label.
    :return: List of pathnames.
    """
    pathnames = []

    for directory in directories:
        subdirectories = glob.glob(os.path.join(directory, "*"))

        pathnames += [glob.glob(os.path.join(subdirectory, "*")) for subdirectory in subdirectories]

    return sum(pathnames, [])


def load(pathnames, labels):
    """
    Load training and target data.
    
    Assumes data is stored in a directory corresponding to some class label.

    :param pathnames: List of image pathnames.
    :param labels: List of class labels.
    :return: Tuple (training, target) data, as NumPy arrays.
    """
    x = numpy.empty((len(pathnames),) + _shape(pathnames[0]), dtype=numpy.uint8)

    y = numpy.empty((len(pathnames),), dtype=numpy.uint8)

    label_to_index = {label: index for index, label in enumerate(sorted(labels))}

    for index, pathname in enumerate(pathnames):
        label = os.path.split(os.path.dirname(pathname))[-1]

        x[index] = numpy.load(pathname)

        y[index] = label_to_index[label]

    return x, y


def _shape(pathname):
    """
    Infer the shape of the sample data from a single sample.
    
    :param pathname: Path to a sample.
    :return: Sample dimensions.
    """
    return numpy.load(pathname).shape

In [ ]:
directories = ["/data/experiment_00", "/data/experiment_00"]

labels = ["class_A", "class_B", "class_C"]

pathnames = collect_pathnames(directories)

x, y = load(pathnames, labels)

The evaluation and target data (`x` and `y`, respectively) is next passed to the model for evaluation. **A previously trained model is required.** The `evaluate` method loads the trained model weights. See the `fit` notebook for instructions on training a model. 

Evaluation data is provided to the model in batches of 32 samples. Use `batch_size` to configure the number of samples. A smaller `batch_size` requires less memory.

The evaluate function outputs the model's loss and accuracy metrics as the array `[loss, accuracy]`.

In [ ]:
model = deepometry.model.Model(shape=x.shape[1:], units=len(numpy.unique(y)))

model.compile()

model.evaluate(x, y, batch_size=32, verbose=1)